In [1]:
# make sure that rpy2 is installed: https://rpy2.bitbucket.io/
%load_ext rpy2.ipython

import warnings
from rpy2.rinterface import RRuntimeWarning
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

# ignore R warning messages
warnings.filterwarnings("ignore", category=RRuntimeWarning)
import anndata

/home/wenhao/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%%R -o N1_DAT  -o geneinfo_N1 -o cellinfo_N1 

load('/home/wenhao/RAW_REAL/RAW_REAL/RAW_SCNORM_H9.RData')
qq<-round(cbind(H9_p24,H9_p96)[whichg_H9,])

cellinfo_N1<-data.frame(Cell=colnames(qq))
geneinfo_N1<-data.frame(Gene=rownames(qq))

N1_DAT<-as.data.frame(t(qq))


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy.api as sc
import DCA.api as dca

Using TensorFlow backend.


In [4]:
adata_N1 = sc.AnnData(N1_DAT.values,var=geneinfo_N1,obs=cellinfo_N1)
adata_N1.var_names = geneinfo_N1.Gene
adata_N1.obs_names = cellinfo_N1.Cell
adata_N1


AnnData object with n_obs × n_vars = 182 × 13195 
    obs: 'Cell'
    var: 'Gene'

In [5]:
%%time

res_N1 = dca.autoencode(adata_N1, training_kwargs={'verbose': False})


### Autoencoder: Successfully preprocessed 13195 genes and 182 cells.
Calculating low dimensional representations...
Calculating reconstructions...
CPU times: user 3min 20s, sys: 1min 43s, total: 5min 3s
Wall time: 56.9 s


In [6]:
N1= res_N1.obsm['X_dca']
nr,nc = N1.shape
Br = ro.r.matrix(N1, nrow=nr, ncol=nc)
ro.r.assign("DCA_H9", Br)

5.555995,7.899257,4.760893,5.411572,...,48.698837,35.011845,65.336800,44.170856


In [7]:
%%R
DCA_H9<-t(DCA_H9)
rownames(DCA_H9)<-geneinfo_N1[,1]
colnames(DCA_H9)<-cellinfo_N1[,1]

save(DCA_H9,file='/home/wenhao/RAW_REAL/DCA_norm/DCA_H9.RData')